# Notebook for Kaggle Competition: Open Problems Single Cell Perturbations 
# Team Yellow Avocado

In [ ]:
Note: The code for data exploration & visualization, internal cross-validation, feature selection, model selection, and unsubmitted methods, and approach that doesn't lead to higher score are not included to avoid unnecessary lengthy code. 

## Import libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
from pathlib import Path
import statistics
import zipfile
from itertools import product
from statistics import mean
import collections

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import VotingRegressor
from sklearn.neural_network import MLPRegressor

## Download data

In [13]:
path = Path("open-problems-single-cell-perturbations")

In [14]:
!kaggle competitions download {path}

100%|█████████████████████████████████████▉| 3.83G/3.83G [02:09<00:00, 31.9MB/s]
100%|██████████████████████████████████████| 3.83G/3.83G [02:09<00:00, 31.8MB/s]


In [15]:
zipfile.ZipFile(f"{path}.zip").extractall(path)

In [24]:
!mv {path}/* .

In [25]:
ls

LICENSE
README.md
adata_excluded_ids.csv
adata_obs_meta.csv
adata_train.parquet
avocado.ipynb
avocado_clean_notebook.ipynb
de_train.parquet
id_map.csv
multiome_obs_meta.csv
multiome_train.parquet
multiome_var_meta.csv
open-problems-single-cell-perturbations/
open-problems-single-cell-perturbations.zip
sample_submission.csv


## Load main competition files

In [3]:
de_train = pd.read_parquet("de_train.parquet")

In [4]:
id_map = pd.read_csv("id_map.csv")

## Create global variables

In [5]:
# decide on default expression level when
default_expression = 0

In [6]:
id_map_sm_name_label = id_map.sm_name.unique()

In [7]:
cell_type_label = list(de_train["cell_type"].unique())
print(cell_type_label)

['NK cells', 'T cells CD4+', 'T cells CD8+', 'T regulatory cells', 'B cells', 'Myeloid cells']


In [8]:
sm_name_label = list(de_train["sm_name"].unique())
print(len(sm_name_label))

146


In [9]:
sm_name_B_and_myeloid = list(
    de_train.loc[
        (de_train.cell_type == "B cells") | (de_train.cell_type == "Myeloid cells")
    ]["sm_name"].unique()
)

In [10]:
sm_name_to_be_test = set(sm_name_label) - set(sm_name_B_and_myeloid)
print(len(sm_name_to_be_test))

129


In [11]:
all_gene = list(de_train.columns)[5:]
print(len(all_gene))

18211


In [12]:
correction_factor = len(all_gene) * len(sm_name_label) * len(cell_type_label)
correction_factor

15952836

In [13]:
math.log10(0.05 / correction_factor)

-8.503867896202365

## Explore missing info

In [14]:
# check missing differential expression info for each cell line
missing_tmp = []
for tmp_cell_type in cell_type_label:
    print(tmp_cell_type)
    missing = set(sm_name_label) - set(
        de_train.loc[(de_train.cell_type == tmp_cell_type)].sm_name.unique()
    )
    print(missing)
    print(len(missing))
    missing_tmp.append(missing)

NK cells
set()
0
T cells CD4+
set()
0
T cells CD8+
{'Resminostat', 'Alvocidib', 'Oprozomib (ONX 0912)', 'CGP 60474'}
4
T regulatory cells
set()
0
B cells
{'GO-6976', 'RG7112', 'Nilotinib', 'CEP-37440', 'Cabozantinib', 'Trametinib', 'Tosedostat', 'Disulfiram', 'Clotrimazole', 'Protriptyline', 'AVL-292', 'Nefazodone', 'Perhexiline', 'BAY 87-2243', 'Bosutinib', 'Canertinib', 'TR-14035', 'RVX-208', 'Sgc-cbp30', 'Ixabepilone', 'K-02288', 'PD-0325901', 'BAY 61-3606', 'TPCA-1', 'Mometasone Furoate', 'HMN-214', 'BMS-777607', 'BMS-536924', 'SCH-58261', 'AT13387', 'CGM-097', 'Topotecan', 'Amiodarone', 'IN1451', 'STK219801', 'AZ628', 'Dovitinib', 'Atorvastatin', 'TL_HRAS26', 'Vandetanib', 'Ketoconazole', 'CGP 60474', 'BX 912', 'AZD4547', 'Raloxifene', 'PRT-062607', 'ABT-199 (GDC-0199)', 'TGX 221', 'I-BET151', 'Vorinostat', 'Imatinib', 'Buspirone', 'Lapatinib', '5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-6-yl)pyrimidin-2-amine', 'BI-D1870', 'LY2090314', 'Sunitinib', 'Desloratadine', 'GLPG0634',

In [15]:
missing_tmp[-1] == missing_tmp[-2]

True

## Data transformation

In [16]:
target_df_columns = [
    "target_cell",
    "sm_name",
    "gene",
    "training",
    "NK cells exp",
    "T cells CD4+ exp",
    "T cells CD8+ exp",
    "T regulatory cells exp",
    "target_averge_expression",
]

In [17]:
all_tergetcell_sm_gene_combination = product(
    ["B cells", "Myeloid cells"], sm_name_label, all_gene
)

In [18]:
all_tergetcell_sm_gene_combination = list(all_tergetcell_sm_gene_combination)

In [19]:
18211 * 146 * 2

5317612

In [20]:
transform_de_train = pd.DataFrame(
    all_tergetcell_sm_gene_combination, columns=target_df_columns[:3]
)

In [21]:
transform_de_train.head(2)

,target_cell,sm_name,gene
0,B cells,Clotrimazole,A1BG
1,B cells,Clotrimazole,A1BG-AS1


In [22]:
in_training = [
    True if i in sm_name_B_and_myeloid else False
    for i in list(transform_de_train["sm_name"])
]

In [23]:
transform_de_train["training"] = in_training

In [24]:
transform_de_train.describe()

,target_cell,sm_name,gene,training
count,5317612,5317612,5317612,5317612
unique,2,146,18211,2
top,B cells,Clotrimazole,A1BG,False
freq,2658806,36422,292,4698438


In [25]:
for i in cell_type_label:
    print(i)
    print(len(de_train.loc[de_train.cell_type == i]["sm_name"].unique()))

NK cells
146
T cells CD4+
146
T cells CD8+
142
T regulatory cells
146
B cells
17
Myeloid cells
17


In [26]:
NKcells_exp = []
TcellsCD4_exp = []
Tregulatorycells_exp = []
for tmp_sm_name in sm_name_label:
    NKcells_exp.extend(
        de_train.loc[
            (de_train["cell_type"] == "NK cells")
            & (de_train["sm_name"] == tmp_sm_name),
            "A1BG":"ZZEF1",
        ]
        .values.flatten()
        .tolist()
    )
    TcellsCD4_exp.extend(
        de_train.loc[
            (de_train["cell_type"] == "T cells CD4+")
            & (de_train["sm_name"] == tmp_sm_name),
            "A1BG":"ZZEF1",
        ]
        .values.flatten()
        .tolist()
    )
    Tregulatorycells_exp.extend(
        de_train.loc[
            (de_train["cell_type"] == "T regulatory cells")
            & (de_train["sm_name"] == tmp_sm_name),
            "A1BG":"ZZEF1",
        ]
        .values.flatten()
        .tolist()
    )

In [27]:
T8_sm_name = list(
    de_train.loc[de_train.cell_type == "T cells CD8+"]["sm_name"].unique()
)
T8_sm_name_missing = list(set(sm_name_label) - set(T8_sm_name))
T8_sm_name_missing

['Resminostat', 'Alvocidib', 'Oprozomib (ONX 0912)', 'CGP 60474']

In [28]:
TcellsCD8_exp = []
for tmp_sm_name in sm_name_label:
    if tmp_sm_name in T8_sm_name_missing:
        TcellsCD8_exp.extend([default_expression] * len(all_gene))
    else:
        TcellsCD8_exp.extend(
            de_train.loc[
                (de_train["cell_type"] == "T cells CD8+")
                & (de_train["sm_name"] == tmp_sm_name),
                "A1BG":"ZZEF1",
            ]
            .values.flatten()
            .tolist()
        )

In [29]:
NKcells_exp = NKcells_exp * 2
TcellsCD4_exp = TcellsCD4_exp * 2
Tregulatorycells_exp = Tregulatorycells_exp * 2
TcellsCD8_exp = TcellsCD8_exp * 2

In [30]:
transform_de_train["NK cells exp"] = NKcells_exp
transform_de_train["T cells CD4+ exp"] = TcellsCD4_exp
transform_de_train["T cells CD8+ exp"] = TcellsCD8_exp
transform_de_train["T regulatory cells exp"] = Tregulatorycells_exp

In [31]:
transform_de_train.shape

(5317612, 8)

In [32]:
transform_de_train

,target_cell,sm_name,gene,training,NK cells exp,T cells CD4+ exp,T cells CD8+ exp,T regulatory cells exp
0,B cells,Clotrimazole,A1BG,False,0.104720,0.915953,-0.387721,0.232893
1,B cells,Clotrimazole,A1BG-AS1,False,-0.077524,-0.884380,-0.305378,0.129029
2,B cells,Clotrimazole,A2M,False,-1.625596,0.371834,0.567777,0.336897
3,B cells,Clotrimazole,A2M-AS1,False,-0.144545,-0.081677,0.303895,0.486946
4,B cells,Clotrimazole,A2MP1,False,0.143555,-0.498266,-0.022653,0.767661
...,...,...,...,...,...,...,...,...
5317607,Myeloid cells,Riociguat,ZXDB,False,1.028394,-0.005004,0.024849,-0.120252
5317608,Myeloid cells,Riociguat,ZXDC,False,0.034144,0.552810,0.012862,-0.064537
5317609,Myeloid cells,Riociguat,ZYG11B,False,-0.231642,-0.209077,-0.029684,-0.603280
5317610,Myeloid cells,Riociguat,ZYX,False,1.023994,0.389751,0.005506,-0.098041


In [33]:
bcell_target_averge_expression = []
myeloid_target_averge_expression = []
for tmp_gene in all_gene:
    bcell_target_averge_expression.append(
        mean(de_train.loc[de_train.cell_type == "B cells"][tmp_gene].tolist())
    )
    myeloid_target_averge_expression.append(
        mean(de_train.loc[de_train.cell_type == "Myeloid cells"][tmp_gene].tolist())
    )

In [34]:
target_averge_expression = bcell_target_averge_expression * len(
    sm_name_label
) + myeloid_target_averge_expression * len(sm_name_label)

In [35]:
len(target_averge_expression)

5317612

In [36]:
transform_de_train["target_averge_expression"] = target_averge_expression

### Remove missing value in B and Myeloid cell


In [37]:
bcell_sm_name_training = list(
    de_train.loc[de_train.cell_type == "B cells"]["sm_name"].unique()
)
bcell_sm_name_test = list(id_map.loc[id_map.cell_type == "B cells"]["sm_name"].unique())
bcell_sm_name_missing = list(
    set(sm_name_label) - set(bcell_sm_name_training) - set(bcell_sm_name_test)
)

In [38]:
bcell_sm_name_missing

['CEP-18770 (Delanzomib)']

In [39]:
myeloid_sm_name_training = list(
    de_train.loc[de_train.cell_type == "Myeloid cells"]["sm_name"].unique()
)
myeloid_sm_name_test = list(
    id_map.loc[id_map.cell_type == "Myeloid cells"]["sm_name"].unique()
)
myeloid_sm_name_missing = list(
    set(sm_name_label) - set(myeloid_sm_name_training) - set(myeloid_sm_name_test)
)

In [40]:
myeloid_sm_name_missing

['BMS-387032', 'CGP 60474']

In [41]:
transform_de_train.shape

(5317612, 9)

In [42]:
transform_de_train.drop(
    transform_de_train[
        (transform_de_train["target_cell"] == "B cells")
        & (transform_de_train["sm_name"] == "CEP-18770 (Delanzomib)")
    ].index,
    inplace=True,
)
transform_de_train.drop(
    transform_de_train[
        (transform_de_train["target_cell"] == "Myeloid cells")
        & (transform_de_train["sm_name"] == "CGP 60474")
    ].index,
    inplace=True,
)
transform_de_train.drop(
    transform_de_train[
        (transform_de_train["target_cell"] == "Myeloid cells")
        & (transform_de_train["sm_name"] == "BMS-387032")
    ].index,
    inplace=True,
)

In [43]:
(5317612 - 5262979) / len(all_gene)

3.0

## Separate test vs trianing+val

In [45]:
training_transform_de_train = transform_de_train.loc[
    transform_de_train["sm_name"].isin(sm_name_B_and_myeloid)
].copy()
test_transform_de_train = transform_de_train.loc[
    transform_de_train["sm_name"].isin(sm_name_to_be_test)
].copy()

In [46]:
print(transform_de_train.shape)
print(training_transform_de_train.shape)
print(test_transform_de_train.shape)

(5262979, 9)
(619174, 9)
(4643805, 9)


### Add response


In [47]:
ordered_myeloid_sm_name_training = training_transform_de_train["sm_name"].unique()
bcell_exp = []
myeloid_exp = []
for tmp_sm_name in ordered_myeloid_sm_name_training:
    bcell_exp.extend(
        de_train.loc[
            (de_train["cell_type"] == "B cells") & (de_train["sm_name"] == tmp_sm_name),
            "A1BG":"ZZEF1",
        ]
        .values.flatten()
        .tolist()
    )
    myeloid_exp.extend(
        de_train.loc[
            (de_train["cell_type"] == "Myeloid cells")
            & (de_train["sm_name"] == tmp_sm_name),
            "A1BG":"ZZEF1",
        ]
        .values.flatten()
        .tolist()
    )

In [48]:
training_transform_de_train["response"] = bcell_exp + myeloid_exp

In [49]:
training_transform_de_train

,target_cell,sm_name,gene,training,NK cells exp,T cells CD4+ exp,T cells CD8+ exp,T regulatory cells exp,target_averge_expression,response
36422,B cells,Idelalisib,A1BG,True,0.861487,0.206471,0.046959,0.210456,1.380890,0.394173
36423,B cells,Idelalisib,A1BG-AS1,True,-0.112313,0.014638,-0.346839,0.842411,0.530585,-0.153824
36424,B cells,Idelalisib,A2M,True,-0.355217,-0.247518,0.023478,0.428241,1.340812,0.178232
36425,B cells,Idelalisib,A2M-AS1,True,0.719999,0.430198,0.485611,0.556238,1.594307,0.566241
36426,B cells,Idelalisib,A2MP1,True,0.655865,0.103020,0.005066,0.792801,4.927551,0.391377
...,...,...,...,...,...,...,...,...,...,...
5080864,Myeloid cells,CHIR-99021,ZXDB,True,0.015837,0.000120,0.037701,0.814064,0.612681,-0.160210
5080865,Myeloid cells,CHIR-99021,ZXDC,True,-0.055027,-1.117706,-0.257844,0.536660,-0.583563,-0.886414
5080866,Myeloid cells,CHIR-99021,ZYG11B,True,-0.329874,-0.130162,-1.374965,-0.108290,-0.427938,-2.955785
5080867,Myeloid cells,CHIR-99021,ZYX,True,0.327199,0.001642,-1.202369,-0.506495,-0.292768,-0.866944


## Run model on full data


In [50]:
training_transform_de_train

,target_cell,sm_name,gene,training,NK cells exp,T cells CD4+ exp,T cells CD8+ exp,T regulatory cells exp,target_averge_expression,response
36422,B cells,Idelalisib,A1BG,True,0.861487,0.206471,0.046959,0.210456,1.380890,0.394173
36423,B cells,Idelalisib,A1BG-AS1,True,-0.112313,0.014638,-0.346839,0.842411,0.530585,-0.153824
36424,B cells,Idelalisib,A2M,True,-0.355217,-0.247518,0.023478,0.428241,1.340812,0.178232
36425,B cells,Idelalisib,A2M-AS1,True,0.719999,0.430198,0.485611,0.556238,1.594307,0.566241
36426,B cells,Idelalisib,A2MP1,True,0.655865,0.103020,0.005066,0.792801,4.927551,0.391377
...,...,...,...,...,...,...,...,...,...,...
5080864,Myeloid cells,CHIR-99021,ZXDB,True,0.015837,0.000120,0.037701,0.814064,0.612681,-0.160210
5080865,Myeloid cells,CHIR-99021,ZXDC,True,-0.055027,-1.117706,-0.257844,0.536660,-0.583563,-0.886414
5080866,Myeloid cells,CHIR-99021,ZYG11B,True,-0.329874,-0.130162,-1.374965,-0.108290,-0.427938,-2.955785
5080867,Myeloid cells,CHIR-99021,ZYX,True,0.327199,0.001642,-1.202369,-0.506495,-0.292768,-0.866944


### Remove low cell count molecule  or with positive ocntrol

In [51]:
training_transform_de_train_tmp = training_transform_de_train[
    ~training_transform_de_train["sm_name"].isin(
        ["Alvocidib", "MLN 2238", "Oprozomib (ONX 0912)", "Belinostat", "Dabrafenib"]
    )
]

In [52]:
training_transform_de_train_tmp.sm_name.unique()

array(['Idelalisib', 'Crizotinib', 'Linagliptin', 'Palbociclib',
       'LDN 193189', 'R428', 'Porcn Inhibitor III', 'Foretinib',
       'Penfluridol', 'Dactolisib', 'O-Demethylated Adapalene',
       'CHIR-99021'], dtype=object)

In [53]:
training_transform_de_train = training_transform_de_train_tmp

### Sep M and B

In [59]:
training_transform_de_train_b = training_transform_de_train[
    training_transform_de_train.target_cell == "B cells"
]
training_transform_de_train_m = training_transform_de_train[
    training_transform_de_train.target_cell == "Myeloid cells"
]
training_transform_de_train_b.reset_index(inplace=True, drop=True)
training_transform_de_train_m.reset_index(inplace=True, drop=True)

In [60]:
test_transform_de_train_b = test_transform_de_train[
    test_transform_de_train.target_cell == "B cells"
]
test_transform_de_train_m = test_transform_de_train[
    test_transform_de_train.target_cell == "Myeloid cells"
]
test_transform_de_train_b.reset_index(inplace=True, drop=True)
test_transform_de_train_m.reset_index(inplace=True, drop=True)

In [61]:
full_training_b = training_transform_de_train_b.drop(
    columns=["response", "target_cell", "sm_name", "gene", "training"]
).copy()
full_testing_b = test_transform_de_train_b.drop(
    columns=["target_cell", "sm_name", "gene", "training"]
).copy()
full_response_b = training_transform_de_train_b["response"]
m1_3 = ElasticNet()
m2_3 = ExtraTreesRegressor(n_estimators=10, max_depth=10, random_state=1)
m6 = MLPRegressor(hidden_layer_sizes=(10,), random_state=1, max_iter=500)

model = VotingRegressor([("lr", m1_3), ("rfr", m2_3)])

model.fit(full_training_b, full_response_b)
y_full_training_pred_b = model.predict(full_training_b)
y_full_testing_pred_b = model.predict(full_testing_b)
print(mean_squared_error(full_response_b, y_full_training_pred_b))

0.6550586676428908


In [62]:
full_training_m = training_transform_de_train_m.drop(
    columns=["response", "target_cell", "sm_name", "gene", "training"]
).copy()
full_testing_m = test_transform_de_train_m.drop(
    columns=["target_cell", "sm_name", "gene", "training"]
).copy()
full_response_m = training_transform_de_train_m["response"]
m1_3 = ElasticNet()
m2_3 = ExtraTreesRegressor(n_estimators=10, max_depth=10, random_state=1)
m6 = MLPRegressor(hidden_layer_sizes=(10,), random_state=1, max_iter=500)

# model=VotingRegressor([('lr', m1_3), ('rfr', m2_3),('nn=10',m6)])
model = VotingRegressor([("lr", m1_3), ("rfr", m2_3)])

# model=
model.fit(full_training_m, full_response_m)
y_full_training_pred_m = model.predict(full_training_m)
y_full_testing_pred_m = model.predict(full_testing_m)
print(mean_squared_error(full_response_m, y_full_training_pred_m))

2.430848804783438


In [63]:
test_transform_de_train_b

,target_cell,sm_name,gene,training,NK cells exp,T cells CD4+ exp,T cells CD8+ exp,T regulatory cells exp,target_averge_expression
0,B cells,Clotrimazole,A1BG,False,0.104720,0.915953,-0.387721,0.232893,1.380890
1,B cells,Clotrimazole,A1BG-AS1,False,-0.077524,-0.884380,-0.305378,0.129029,0.530585
2,B cells,Clotrimazole,A2M,False,-1.625596,0.371834,0.567777,0.336897,1.340812
3,B cells,Clotrimazole,A2M-AS1,False,-0.144545,-0.081677,0.303895,0.486946,1.594307
4,B cells,Clotrimazole,A2MP1,False,0.143555,-0.498266,-0.022653,0.767661,4.927551
...,...,...,...,...,...,...,...,...,...
2331003,B cells,Riociguat,ZXDB,False,1.028394,-0.005004,0.024849,-0.120252,0.581303
2331004,B cells,Riociguat,ZXDC,False,0.034144,0.552810,0.012862,-0.064537,0.637408
2331005,B cells,Riociguat,ZYG11B,False,-0.231642,-0.209077,-0.029684,-0.603280,0.517737
2331006,B cells,Riociguat,ZYX,False,1.023994,0.389751,0.005506,-0.098041,-0.207092


### Concat response

In [64]:
y_full_testing_pred = list(y_full_testing_pred_b) + list(y_full_testing_pred_m)

## Transform and submit

In [65]:
combine_test = test_transform_de_train.iloc[:, :3].copy()
combine_test.columns = ["cell_type", "sm_name", "gene"]
combine_test["level"] = y_full_testing_pred
combine_test.level.describe()

count    4.643805e+06
mean     2.953019e-01
std      8.491516e-01
min     -1.303315e+01
25%      3.975647e-02
50%      1.288916e-01
75%      2.904235e-01
max      3.387086e+01
Name: level, dtype: float64

In [66]:
sort_combine_test = combine_test.sort_values(by=["cell_type", "sm_name"])

In [67]:
sort_combine_test.sm_name.unique() == id_map.sm_name.unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [68]:
first_line = "id" + "," + ",".join(all_gene)

In [69]:
print(len(bcell_sm_name_test))
print(len(myeloid_sm_name_test))
print(len(all_gene))

128
127
18211


In [70]:
otherline = []
writeout_expression = list(sort_combine_test["level"])
for i in range(255):
    otherline.append(
        str(i)
        + ","
        + ",".join(
            map(str, writeout_expression[i * len(all_gene) : (i + 1) * len(all_gene)])
        )
    )

In [71]:
with open("votem1_3m2_3_pos.csv", "w") as fd:
    fd.writelines(first_line + "\n")
    for line in otherline:
        fd.writelines(line + "\n")